In [3]:
#This script performs centroiding, star identification and attitude detrminatoon for FITS images
import numpy as np
import cv2
from astropy.wcs import WCS
import matplotlib
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils import DAOStarFinder
from photutils import find_peaks
from astropy.visualization import SqrtStretch
from photutils import CircularAperture
from astropy.io import ascii
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.coordinates import FK5
from astropy.table import Column
from scipy import stats
from scipy.special import comb
import math
from PIL import Image
import time
from numba import jit
from pyquaternion import Quaternion
from Quaternion import Quat
from Quaternion import normalize
from math import cos, sin, radians, degrees, atan2, sqrt

%matplotlib qt 

#****************************************************Geometric Voting**********************************************************

#Camera specific variables
fl = 85                     #focal length of imager in mm
ps = 20.11                  #pixel scale in arcsec/pixel
p_pitch = (ps/206265)*fl    #pixel pitch of imager, assuming square pixel
coeff = p_pitch/fl
princ_x = 918               #x coordinate of principal point 
princ_y = 1375              #y coordinate of principal point

#Program variables
u = 0.00025                 #allocated angular error
star_guess = []
peak = 0

cat_dist = ascii.read("cat_dist.dat")

#Binary search function
@jit
def binary_search(array, target, dist_min, dist_max ):
    lower = 0
    upper = len(cat_dist)
    while (lower < upper):   
        x = lower + (upper - lower) // 2
        val = cat_dist['ang_dist'][x]
        if (val > dist_min) & (val < dist_max):
            return(x)
        elif (target > val):
            if (lower == x):   
                break        
            lower = x
        elif (target < val):
            upper = x

#Read in FITS image and store primary data 
hdu_list = fits.open('Del Cep 2016 G0716 G_0002.fit')
image_data = hdu_list[0].data


#Combined Peak and Source Centroiding Methods
mean, median, std = sigma_clipped_stats(image_data, sigma=3.0, iters=5)    
daofind = DAOStarFinder(fwhm=5.0, threshold=5.*std)    
sources = daofind(image_data - median)    
positions = (sources['xcentroid'], sources['ycentroid'])

if len(sources)<5:
    threshold = median + (1 * std)
    sources = find_peaks(image_data, threshold, box_size=5)   
    positions = (sources['x_peak'], sources['y_peak'])
    peak = 1    
    
#Use only 15 brightest stars
if len(sources)>15:
    if peak == 0:
        mask = sources['flux'].argsort()[-15:][::-1]
        sources = sources[mask]
    else:
        mask = sources['peak_value'].argsort()[-15:][::-1]
        sources = sources[mask]


#Pre-assign table for unit vectors of stars in image
N = len(sources)
dtype = [('star', 'i4'), ('x', 'f8'), ('y', 'f8'), ('z', 'f8')]
star_eci = Table(data=np.zeros(N, dtype=dtype))


#Create voting table 
votes= Table()
star_no = 0
rows = (comb(len(sources), 2, exact=True))*1000
bb = Column(np.full((rows, 1), 0) )
for n in range(len(sources)):
    votes.add_column(bb, name = star_no)
    star_no = star_no+1
    
    
#Create index array to keep track of votes for each star
LIST_LENGTH = len(sources)
index = []
while len(index) < LIST_LENGTH:
    index.append(0)

    
#covert pixel coordinates to body-fixed unit vectors 
for i in range(len(sources)):
    if peak == 1:
        star_eci[i] = (i,((sources['x_peak'][i]-princ_x)*coeff)*(math.pow(((1+((math.pow((sources['x_peak'][i]-princ_x),2))+(math.pow((sources['y_peak'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)),((sources['y_peak'][i]-princ_y)*coeff)*(math.pow(((1+((math.pow((sources['x_peak'][i]-princ_x),2))+(math.pow((sources['y_peak'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)),(math.pow(((1+((math.pow((sources['x_peak'][i]-princ_x),2))+(math.pow((sources['y_peak'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)))
    else:
        star_eci[i] = (sources['id'][i],((sources['xcentroid'][i]-princ_x)*coeff)*(math.pow(((1+((math.pow((sources['xcentroid'][i]-princ_x),2))+(math.pow((sources['ycentroid'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)),((sources['ycentroid'][i]-princ_y)*coeff)*(math.pow(((1+((math.pow((sources['xcentroid'][i]-princ_x),2))+(math.pow((sources['ycentroid'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)),(math.pow(((1+((math.pow((sources['xcentroid'][i]-princ_x),2))+(math.pow((sources['ycentroid'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)))


#Modified binary search and voting for each star in sources
for i in range(len(sources)-1):
    for j in range(i+1, len(sources)):
        star_dist = math.acos(((star_eci['x'][i])*(star_eci['x'][j]))+((star_eci['y'][i])*(star_eci['y'][j]))+((star_eci['z'][i])*(star_eci['z'][j])))
        if (star_dist != 0):
            dist_min = star_dist-u
            dist_max = star_dist+u
            k = binary_search(cat_dist['ang_dist'],star_dist, dist_min, dist_max)
            
            #left search
            match_index= k
            while (cat_dist['ang_dist'][match_index] > dist_min):
                votes[index[i]][i] = cat_dist['star_1_id'][match_index]
                votes[(index[i])+1][i] = cat_dist['star_2_id'][match_index]
                index[i] = index[i]+2
                votes[index[j]][j] = cat_dist['star_1_id'][match_index]
                votes[(index[j])+1][j] = cat_dist['star_2_id'][match_index]
                index[j] = index[j]+2
                match_index = match_index-1
                
            #Right search
            match_index = k
            while (cat_dist['ang_dist'][match_index] < dist_max):
                votes[index[i]][i] = cat_dist['star_1_id'][match_index]
                votes[(index[i])+1][i] = cat_dist['star_2_id'][match_index]
                index[i] = index[i]+2
                votes[index[j]][j] = cat_dist['star_1_id'][match_index]
                votes[(index[j])+1][j] = cat_dist['star_2_id'][match_index]
                index[j] = index[j]+2
                match_index = match_index+1
                

#Assign each candidate star to catalogue star with most votes
for a in range(len(sources)):
    zeroes = rows - ((list(votes[a][:])).count(0))
    votes_subset = votes[0:zeroes]
    guess = int(stats.mode(votes_subset[a][:])[0])
    star_guess.append(guess)
    
    
print('Hipparcos IDs of stars in image (initial star matching)'+'\n', star_guess)

#********************************************************Verification**********************************************************
#Verification of initial star guess

final_ver = []
eci_data = ascii.read("star_list.dat")

#initialise voting list to 0
while len(final_ver) < LIST_LENGTH:
    final_ver.append(0)

#Perform verification based on whether angular distance between the candidate stars are similar to 
#angular distance of their matched catalogue star

for i in range(len(sources)-1):
    for j in range(i+1, len(sources)):
        star_dist = math.acos(((star_eci['x'][i])*(star_eci['x'][j]))+((star_eci['y'][i])*(star_eci['y'][j]))+((star_eci['z'][i])*(star_eci['z'][j])))
        dist_min = star_dist-u
        dist_max = star_dist+u
        i_index = (list(eci_data['star_id'])).index(star_guess[i])
        j_index = (list(eci_data['star_id'])).index(star_guess[j])
        act_dist = math.acos(((eci_data['x'][i_index])*(eci_data['x'][j_index]))+((eci_data['y'][i_index])*(eci_data['y'][j_index]))+((eci_data['z'][i_index])*(eci_data['z'][j_index])))
        if (act_dist > star_dist - u) & (act_dist < star_dist+u):
            final_ver[i] = final_ver[i]+1
            final_ver[j] = final_ver[j]+1            

print('Number of votes received per star:'+'\n',final_ver)

#Discard any candidate star which received less than the maximum number of votes
for i in (range(len(final_ver))):
    if final_ver[i] != max(final_ver):
        star_guess[i] = 0
        
print('Non-zero elements correspond to verified Hipparcos IDs of image stars:'+'\n',star_guess)

#**************************************************Display Annotated Image******************************************************
#Display image with Hipparcos identifier number of stars and (RA,DEC) of frame centre

ra_dec = ascii.read("ra_dec.dat")
ra_centre_array = []
dec_centre_array = []

font = {'family': 'serif',
        'color':  'darkgreen',
        'weight': 'normal',
        'size': 8,
        }

for i in (range(len(final_ver))):
    if star_guess[i] == 0:
        positions_false = (sources['xcentroid'][i],sources['ycentroid'][i])
        apertures = CircularAperture(positions_false, r=4.)
        apertures.plot(color='red', lw=1.5, alpha=0.5)

for i in (range(len(final_ver))):
    if star_guess[i] != 0:
        positions_true = (sources['xcentroid'][i], sources['ycentroid'][i])
        apertures_true = CircularAperture(positions_true, r=4.)
        apertures_true.plot(color='blue', lw=1.5, alpha=0.5)
        plt.text(sources['xcentroid'][i]-80, sources['ycentroid'][i]-50, star_guess[i], fontdict=font)
        
        
#Calculate the celestial coordinates of the frame centre (pointing)
for e in range(len(star_guess)):
    if star_guess[e] != 0:
        rd_index = (list(ra_dec['star_id'])).index(star_guess[e])
        ra = ra_dec['ra'][rd_index]
        dec = ra_dec['dec'][rd_index]
        x_pos = sources[e]['xcentroid']
        y_pos = sources[e]['ycentroid']
        ra_centre = round((((x_pos - image_data.shape[1]/2)*ps)/3600 + ra),3)
        dec_centre = round((((y_pos - image_data.shape[0]/2)*ps)/3600 + dec),3)
        ra_centre_array.append(ra_centre)
        dec_centre_array.append(dec_centre)
        
ra_centre = round((np.median(ra_centre_array)),3)
dec_centre = round((np.median(dec_centre_array)),3)

text = 'RA: ' + str(ra_centre) + '; Dec: ' + str(dec_centre)

font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 10,
        }

plt.plot(image_data.shape[1]/2, image_data.shape[0]/2, ls='none', color='darkred', marker='+', ms=10, lw=2.0)
plt.text(image_data.shape[1]/2-(image_data.shape[1]/6.5), image_data.shape[0]/2-50, text, fontdict=font)

# norm = ImageNormalize(stretch=SqrtStretch())
# plt.imshow(image_data, cmap='Greys', norm=norm)


#****************************************************Attitude Determination****************************************************
#Implementation of the TRIAD algorithm

eci_data = ascii.read("star_list.dat")

#Select two matched stars with max votes
for e in range(len(star_guess)):
    if star_guess[e] != 0:
        match1 = star_guess[e]
        index_1 = e
        break 

for e in range (index_1+1,len(star_guess)):
    if star_guess[e] != 0:
        match2 = star_guess[e]
        index_2 = e
        break

#Obtain the ECI unit vectors of their corresponding catalogue stars        
cat_index_1 = (list(eci_data['star_id'])).index(match1)
cat_index_2 = (list(eci_data['star_id'])).index(match2)


#TRIAD algorithm 
v_1_b = np.array([star_eci['x'][index_1],star_eci['y'][index_1],star_eci['z'][index_1]])
v_1_i = np.array([eci_data['x'][cat_index_1],eci_data['y'][cat_index_1], eci_data['z'][cat_index_1]])
v_2_b = np.array([star_eci['x'][index_2],star_eci['y'][index_2],star_eci['z'][index_2]])
v_2_i = np.array([eci_data['x'][cat_index_2],eci_data['y'][cat_index_2], eci_data['z'][cat_index_2]])

t_1_b = v_1_b
t_1_i =  v_1_i

cross_2_b = (np.cross(v_1_b, v_2_b))
t_2_b = (cross_2_b)/(math.sqrt(math.pow(cross_2_b[0],2)+math.pow(cross_2_b[1],2)+math.pow(cross_2_b[2],2)))
cross_2_i = (np.cross(v_1_i, v_2_i))
t_2_i = (cross_2_i)/(math.sqrt(math.pow(cross_2_i[0],2)+math.pow(cross_2_i[1],2)+math.pow(cross_2_i[2],2)))

t_3_b = (np.cross(t_1_b, t_2_b))
t_3_i = (np.cross(t_1_i, t_2_i))

r_b = np.matrix([t_1_b, t_2_b, t_3_b])
r_b = r_b.T
r_i = np.matrix([t_1_i, t_2_i, t_3_i])
r_i = r_i.T

r_matrix = r_b*r_i.T
R = r_matrix
print('triad matrix: ' + '\n', r_matrix)

#--------------------------------Conversion to Euler Angles-------------------------------------

# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6
 
    
# Calculates rotation matrix to euler angles
# Order XYZ 
def rotationMatrixToEulerAngles(R) :
 
    assert(isRotationMatrix(R))
     
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        x = x*(180/math.pi)
        y = math.atan2(-R[2,0], sy)
        y = y*(180/math.pi)
        z = math.atan2(R[1,0], R[0,0])
        z = z*(180/math.pi)
    else :
        x = math.atan2(-R[1,2], R[1,1])
        x = x*(180/math.pi)
        y = math.atan2(-R[2,0], sy)
        y = y*(180/math.pi)
        z = 0
        z = z*(180/math.pi)
 
    return np.array([x, y, z])

angles = rotationMatrixToEulerAngles(R)
print(angles)

#--------------------------------Conversion to Quaternion-------------------------------------

quat = Quaternion(matrix=R)
quat2 = [quat[0], quat[1], quat[2], quat[3]]

#--------------------------------Conversion to RA and DEC-------------------------------------
# Copyright (c) 2010, Smithsonian Astrophysical Observatory
# All rights reserved.
##
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
# * Redistributions of source code must retain the above copyright
# notice, this list of conditions and the following disclaimer.
# * Redistributions in binary form must reproduce the above copyright
# notice, this list of conditions and the following disclaimer in the
# documentation and/or other materials provided with the distribution.
# * Neither the name of the Smithsonian Astrophysical Observatory nor the
# names of its contributors may be used to endorse or promote products
# derived from this software without specific prior written permission.
##
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
# ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL <COPYRIGHT HOLDER> BE LIABLE FOR ANY
# DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
# (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
# LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
# ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
# (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
# SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

#Normalise the quaternion
quat2 = quat2 / np.sqrt(np.dot(quat2, quat2))
quat2 = Quat(quat2)

#Convert quaternion to celestial coordinates
new_ra = quat2.ra
new_dec = quat2.dec
new_roll = quat2.roll

print(round((new_ra),4),' ', round((new_dec),4), ' ', round((new_roll),4))


Hipparcos IDs of stars in image (initial star matching)
 [109492, 37819, 107259, 110991, 107418, 108917, 110538, 42535, 106801, 109556, 109017, 107930, 109572, 111797, 108317]
Number of votes received per star:
 [12, 0, 12, 12, 12, 11, 11, 0, 12, 12, 12, 12, 12, 12, 12]
Non-zero elements correspond to verified Hipparcos IDs of image stars:
 [109492, 0, 107259, 110991, 107418, 0, 0, 0, 106801, 109556, 109017, 107930, 109572, 111797, 108317]
triad matrix: 
 [[-0.57514452 -0.8135226   0.08596375]
 [ 0.66972909 -0.52859987 -0.52157944]
 [ 0.46975709 -0.24241114  0.84886107]]
[-15.93786979 -28.01852956 130.65505725]
344.0621   -28.0185   49.3449
